In [1]:
import psycopg2
from psycopg2 import sql
from psycopg2.extensions import adapt, register_adapter, AsIs
from sqlalchemy import create_engine
import getpass
import pandas as pd
import numpy as np

import pandas as pd
import geopandas as gpd
import os
import webbrowser
import numpy as np
from shapely import wkt
from datetime import datetime

In [ ]:
database = 'f24t03'
user = 'jsmm8'
password = getpass.getpass("Type password and hit enter: ")

Type password and hit enter: ········


In [ ]:
try:
    # Connect to the PostgreSQL database
    connection = psycopg2.connect(dbname=database, user=user, password=password, host='pgsql')
    cursor = connection.cursor()

    # Execute a query to get all table names
    cursor.execute("""
        SELECT table_name 
        FROM information_schema.tables 
        WHERE table_schema = 'public';
    """)
    
    # Fetch all results
    tables = cursor.fetchall()

    # Print the table names
    print("Tables in the database:")
    for table in tables:
        print(table[0])

except Exception as e:
    print(f"An error occurred: {e}")

finally:
    # Close the cursor and connection
    if cursor:
        cursor.close()
    if connection:
        connection.close()

Tables in the database:
ethnicity
year_built
springfield_fire
education_levels
springfield_middle_school
tenure_by_bedrooms
tenure_by_kitchen
tenure_by_plumbing
rooms
imputed_median_house_value
raster_columns
raster_overviews
springfield_police
education_categories
demographics_master
model_features_data
year_structure_built
springfield_early_childhood_school
mortgage
tenure_by_rooms
income_data_processed
race_data_processed
housing_stats_data
race_migration_data
merged_new_house_data
income_pct_change
bedrooms
distance_drive_times
educational_attainment
employment_status
springfield_neighborhood_service_areas
parcel_boundaries
springfield_final_dataset
geo_mobility
home_value
house_heating_fuel
household_income
housing_pct_change_stats_data
internet_subscription
income_to_poverty_ratio
median_home_value
median_household_income
median_rooms_by_tenure
occupancy_status
plumbing_facilities
poverty_status
mortgage_status
springfield_single_family_point
springfield_elementary_school
springf

In [ ]:
# Full data

# === Step 1: Connect to your database ===
conn = psycopg2.connect(database=database, user=user, host='pgsql', password=password)

# === Step 2: Load full table using pandas ===
query = "SELECT * FROM georealedge_data;"
df = pd.read_sql(query, conn)
conn.close()

# === Step 3: Identify all geometry columns ===
geometry_cols = [col for col in df.columns if 'geolocation' in col]

# === Step 4: Convert all geometry columns from WKB to Shapely Points ===
from shapely import wkb

def safe_parse(val):
    try:
        if isinstance(val, str):
            return wkb.loads(val, hex=True)
        elif isinstance(val, (bytes, memoryview)):
            return wkb.loads(val)
    except:
        return None

for col in geometry_cols:
    df[col] = df[col].apply(safe_parse)

# === Step 5: Wrap as GeoDataFrame using one geometry column ===
df_full = gpd.GeoDataFrame(df, geometry='home_geolocation', crs='EPSG:26915')

df_full.head()

,home_geolocation,block_group_id,address,objectid,zip5,nsa,bds_zone,zoning,drive_time_fire,drive_distance_fire,...,square_footage,lot_size,year_built,sinkhole_house_risk_rating,sinkhole_risk_label,has_sinkhole_impact,pct_fld_oc,has_floodplain_impact,floodplain_house_risk_rating,floodplain_risk_label
0,POINT (471174.694 4120899.099),1500000US290770033003,2511 W ATLANTIC ST,6720,65803,TOM WATKIN,A,R-SF,2.75,0.92,...,2358,10688.0,2019,0,No Risk,N,0.0,N,0,No Risk
1,POINT (472875.801 4120913.727),1500000US290770019001,2027 N JOHNSTON AVE,52747,65803,WOODLAND,A,R-SF,3.30,0.98,...,2947,19842.0,1971,0,No Risk,N,0.0,N,0,No Risk
2,POINT (473810.378 4120914.316),1500000US290770019002,2031 N LYON AVE,56844,65803,WOODLAND,BN,R-SF,1.89,0.54,...,3388,19634.0,1979,0,No Risk,N,0.0,N,0,No Risk
3,POINT (471548.433 4120914.286),1500000US290770033003,2014 N PARK AVE,38608,65803,TOM WATKIN,A,R-SF,3.17,1.14,...,2832,21306.0,2014,0,No Risk,N,0.0,N,0,No Risk
4,POINT (474571.749 4120917.463),1500000US290770055003,2040 N BENTON AVE,5218,65803,WOODLAND,BN,R-SF,3.22,0.92,...,2594,10017.0,1962,0,No Risk,N,0.0,N,0,No Risk


In [5]:
# Drop rows with missing values in the three stratifying columns
df_valid = df_full[
    df_full[['nsa', 'has_sinkhole_impact', 'has_floodplain_impact']].notnull().all(axis=1)
]

# Sampling function for each NSA group
def triple_stratified_sample(group):
    total_target = 100
    
    # Group by both sinkhole and floodplain impact
    strat_group = group.groupby(['has_sinkhole_impact', 'has_floodplain_impact'])
    
    # Count samples per subgroup
    subgroup_counts = strat_group.size().reset_index(name='count')
    
    if subgroup_counts.empty:
        return pd.DataFrame()
    
    # Determine proportions
    subgroup_counts['proportion'] = subgroup_counts['count'] / subgroup_counts['count'].sum()
    subgroup_counts['sample_size'] = (subgroup_counts['proportion'] * total_target).round().astype(int)
    
    sampled_rows = []
    
    for _, row in subgroup_counts.iterrows():
        y_n = row['has_sinkhole_impact']
        f_n = row['has_floodplain_impact']
        n_sample = row['sample_size']
        
        sub_df = group[
            (group['has_sinkhole_impact'] == y_n) &
            (group['has_floodplain_impact'] == f_n)
        ]
        
        if len(sub_df) == 0:
            continue
        
        sample = sub_df.sample(n=n_sample, replace=(len(sub_df) < n_sample), random_state=42)
        sampled_rows.append(sample)
    
    return pd.concat(sampled_rows)

# Apply the sampling to each NSA
df_final_sample = df_valid.groupby('nsa', group_keys=False).apply(triple_stratified_sample)

In [6]:
df_final_sample

,home_geolocation,block_group_id,address,objectid,zip5,nsa,bds_zone,zoning,drive_time_fire,drive_distance_fire,...,square_footage,lot_size,year_built,sinkhole_house_risk_rating,sinkhole_risk_label,has_sinkhole_impact,pct_fld_oc,has_floodplain_impact,floodplain_house_risk_rating,floodplain_risk_label
32630,POINT (476953.017 4117407.434),1500000US290770023003,1919 E CHERRYSTREET CT,44094,65802,BINGHAM,C,R-SF,2.87,0.93,...,2778,15158.0,2020,0,No Risk,N,0.0,N,0,No Risk
31894,POINT (476967.327 4117275.622),1500000US290770023003,1913 E MONROE ST,44084,65802,BINGHAM,C,R-SF,3.60,1.10,...,4000,17968.0,2006,0,No Risk,N,0.0,N,0,No Risk
34670,POINT (478872.470 4117847.950),1500000US290770023002,314 S KEN AVE,50604,65802,BINGHAM,C,R-SF,3.14,1.19,...,2741,14677.0,1981,0,No Risk,N,0.0,N,0,No Risk
30629,POINT (477198.535 4117085.964),1500000US290770023003,747 S GRANDVIEW AVE,43861,65802,BINGHAM,C,R-SF,3.30,1.02,...,3915,18973.0,1983,0,No Risk,N,0.0,N,0,No Risk
33734,POINT (478708.373 4117642.303),1500000US290770023002,426 S MONTE VISTA AVE,69580,65802,BINGHAM,C,R-SF,3.67,1.26,...,2157,18491.0,1963,0,No Risk,N,0.0,N,0,No Risk
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5179,POINT (478581.566 4106534.401),1500000US290770040021,5917 S AUDREY CT,101166,65804,ZONE4,C,R-SF,0.80,0.22,...,2015,19946.0,2015,0,No Risk,N,0.0,N,0,No Risk
8255,POINT (474394.251 4110128.349),1500000US290770040041,4231 S KIMBROUGH AVE,31558,65810,ZONE4,BS,R-SF,4.19,1.75,...,3699,13796.0,1967,0,No Risk,N,0.0,N,0,No Risk
19330,POINT (480543.545 4114431.966),1500000US290770038022,3665 E BOWMAN ST,30132,65809,ZONE4,C,R-SF,1.62,0.57,...,3220,17238.0,2004,0,No Risk,N,0.0,N,0,No Risk
15262,POINT (481068.078 4113411.671),1500000US290770038023,3977 E WILSHIRE DR,38517,65809,ZONE4,C,R-SF,0.05,0.01,...,3469,20834.0,1961,0,No Risk,N,0.0,N,0,No Risk


In [7]:
df = df_final_sample.copy()

### Piecewise Model

In [8]:
# piecewise model

def piecewise_score(distance, threshold):
    """Calculate score based on distance from threshold"""
    if distance <= threshold:
        return 1.00
    elif distance <= threshold + 1:
        return 0.85
    elif distance <= threshold + 2:
        return 0.70
    elif distance <= threshold + 3:
        return 0.55
    elif distance <= threshold + 4:
        return 0.40
    else:
        return 0.25

def main():
    print(f"Successfully loaded data with {len(df)} records")
    
    use_filters = input("Do you want to apply filters? (y/n): ").lower() == 'y'
    
    if use_filters:
        min_price = float(input("Enter minimum price (0): ") or 0)
        max_price = float(input("Enter maximum price (1000000): ") or 1000000)
        
        bedrooms_input = input("Enter number of bedrooms (or press Enter for any): ")
        bedrooms = int(bedrooms_input) if bedrooms_input.strip() else None
        
        bathrooms_input = input("Enter number of bathrooms (or press Enter for any): ")
        bathrooms = float(bathrooms_input) if bathrooms_input.strip() else None
        
        min_sqft = float(input("Enter minimum square footage (0): ") or 0)
        max_sqft = float(input("Enter maximum square footage (4500): ") or 10000)
        
        filtered_df = df.copy()
        
        filtered_df = filtered_df[
            (filtered_df['house_price'] >= min_price) & 
            (filtered_df['house_price'] <= max_price) &
            (filtered_df['square_footage'] >= min_sqft) & 
            (filtered_df['square_footage'] <= max_sqft)
        ]
        
        if bedrooms is not None:
            filtered_df = filtered_df[filtered_df['bedrooms'] == bedrooms]
            
        if bathrooms is not None:
            filtered_df = filtered_df[filtered_df['bathrooms'] == bathrooms]
        
        print(f"After filtering: {len(filtered_df)} records remain")
    else:
        filtered_df = df.copy()
    
    distance_columns = [col for col in filtered_df.columns if 'drive_distance_' in col]
    
    print("\nAvailable Points of Interest:")
    for i, col in enumerate(distance_columns, 1):
        poi_name = col.replace('drive_distance_', '')
        print(f"{i}. {poi_name}")
    
    poi_preferences = []
    
    while True:
        choice = int(input("\nSelect a POI (1-{0}) or 0 to finish: ".format(len(distance_columns))))
        if choice == 0:
            break
        
        if 1 <= choice <= len(distance_columns):
            selected_column = distance_columns[choice-1]
            importance = int(input(f"Enter importance (1-5) for {selected_column.replace('drive_distance_', '')}: "))
            distance = float(input(f"Enter maximum preferred distance in miles: "))
            
            poi_preferences.append({
                'column': selected_column,
                'importance': importance,
                'distance': distance
            })
            print(f"Added {selected_column.replace('drive_distance_', '')}")
        else:
            print("Invalid choice")
    
    if not poi_preferences:
        print("No POIs selected. Exiting.")
        return
    
    scored_df = filtered_df.copy()
    
    selected_pois = [poi['column'] for poi in poi_preferences]
    thresholds = {poi['column']: poi['distance'] for poi in poi_preferences}
    importance = {poi['column']: poi['importance'] for poi in poi_preferences}
    
    total_importance = sum(importance.values())
    weights = {poi: val / total_importance for poi, val in importance.items()}
    
    for poi in selected_pois:
        score_column = f"{poi}_score"
        scored_df[score_column] = scored_df[poi].apply(
            lambda x: piecewise_score(x, thresholds[poi])
        )
    
    scored_df["final_score"] = scored_df.apply(
        lambda row: sum(
            row[f"{poi}_score"] * weights[poi] for poi in selected_pois if f"{poi}_score" in row
        ),
        axis=1
    )
    
    scored_df = scored_df.sort_values("final_score", ascending=False)
    
    #num_results = int(input("How many top results would you like to see? "))
    
    print("\n===== MATCHING PROPERTIES =====")
    base_columns = ["address", "house_price", "bedrooms", "bathrooms", "square_footage", "final_score"]
    distance_display_cols = selected_pois  # show the raw distances to selected POIs
    display_columns = base_columns + distance_display_cols
    
    print(scored_df[display_columns].round(2))

if __name__ == "__main__":
    main()

Successfully loaded data with 3600 records
Do you want to apply filters? (y/n): y
Enter minimum price (0): 40000
Enter maximum price (1000000): 750000
Enter number of bedrooms (or press Enter for any): 3
Enter number of bathrooms (or press Enter for any): 2
Enter minimum square footage (0): 0
Enter maximum square footage (4500): 4500
After filtering: 133 records remain

Available Points of Interest:
1. fire
2. police
3. early_childhood_school
4. elementary_school
5. middle_school
6. high_school
7. hospital
8. nursing_home
9. public_health
10. urgentcare
11. trailheads

Select a POI (1-11) or 0 to finish: 7
Enter importance (1-5) for hospital: 5
Enter maximum preferred distance in miles: 3
Added hospital

Select a POI (1-11) or 0 to finish: 11
Enter importance (1-5) for trailheads: 4
Enter maximum preferred distance in miles: 2
Added trailheads

Select a POI (1-11) or 0 to finish: 0

===== MATCHING PROPERTIES =====
                    address  house_price  bedrooms  bathrooms  square_fo

### TOPSIS Model

In [10]:
# topsis model

def normalize_matrix(matrix):
    return matrix / np.sqrt((matrix ** 2).sum())


def topsis_score(df, criteria_columns, weights):
    decision_matrix = df[criteria_columns].copy()
    norm_matrix = normalize_matrix(decision_matrix)
    weighted_matrix = norm_matrix * weights

    pis = weighted_matrix.min()  
    nis = weighted_matrix.max()

    d_pos = np.sqrt(((weighted_matrix - pis) ** 2).sum(axis=1))
    d_neg = np.sqrt(((weighted_matrix - nis) ** 2).sum(axis=1))
    scores = d_neg / (d_pos + d_neg)

    return scores

def main():
    df = pd.read_csv("full_db.csv")
    print(f"Successfully loaded data with {len(df)} records")

    use_filters = input("Do you want to apply filters? (y/n): ").lower() == 'y'
    if use_filters:
        min_price = float(input("Enter minimum price (0): ") or 0)
        max_price = float(input("Enter maximum price (1000000): ") or 1000000)

        bedrooms_input = input("Enter number of bedrooms (or press Enter for any): ")
        bedrooms = int(bedrooms_input) if bedrooms_input.strip() else None

        bathrooms_input = input("Enter number of bathrooms (or press Enter for any): ")
        bathrooms = float(bathrooms_input) if bathrooms_input.strip() else None

        min_sqft = float(input("Enter minimum square footage (0): ") or 0)
        max_sqft = float(input("Enter maximum square footage (4500): ") or 10000)

        filtered_df = df[
            (df['house_price'] >= min_price) &
            (df['house_price'] <= max_price) &
            (df['square_footage'] >= min_sqft) &
            (df['square_footage'] <= max_sqft)
        ]

        if bedrooms is not None:
            filtered_df = filtered_df[filtered_df['bedrooms'] == bedrooms]

        if bathrooms is not None:
            filtered_df = filtered_df[filtered_df['bathrooms'] == bathrooms]

        print(f"After filtering: {len(filtered_df)} records remain")
    else:
        filtered_df = df.copy()

    distance_columns = [col for col in filtered_df.columns if 'drive_distance_' in col]

    print("\nAvailable Points of Interest:")
    for i, col in enumerate(distance_columns, 1):
        print(f"{i}. {col.replace('drive_distance_', '')}")

    poi_preferences = []

    while True:
        choice = int(input("\nSelect a POI (1-{0}) or 0 to finish: ".format(len(distance_columns))))
        if choice == 0:
            break

        if 1 <= choice <= len(distance_columns):
            selected_column = distance_columns[choice - 1]
            importance = int(input(f"How important is it for this POI ({selected_column.replace('drive_distance_', '')}) to be CLOSE to the property? (1-5): "))

            poi_preferences.append({
                'column': selected_column,
                'importance': importance
            })
            print(f"Added {selected_column.replace('drive_distance_', '')}")
        else:
            print("Invalid choice")

    if not poi_preferences:
        print("No POIs selected. Exiting.")
        return

    scored_df = filtered_df.copy()

    selected_pois = [poi['column'] for poi in poi_preferences]
    importance = {poi['column']: poi['importance'] for poi in poi_preferences}

    total_importance = sum(importance.values())
    weights = np.array([importance[poi] / total_importance for poi in selected_pois])

    # Calculate TOPSIS scores
    scored_df["topsis_score"] = topsis_score(scored_df, selected_pois, weights)
    scored_df = scored_df.sort_values("topsis_score", ascending=False)

    print("\n===== PROPERTIES RANKED BY TOPSIS SCORE =====")
    display_columns = ["address", "house_price", "bedrooms", "bathrooms", "square_footage"] + selected_pois + ["topsis_score"]
    print(scored_df[display_columns].round(2).to_string(index=False))

if __name__ == "__main__":
    main()

Successfully loaded data with 3600 records
Do you want to apply filters? (y/n): y
Enter minimum price (0): 0
Enter maximum price (1000000): 750000
Enter number of bedrooms (or press Enter for any): 3
Enter number of bathrooms (or press Enter for any): 2
Enter minimum square footage (0): 0
Enter maximum square footage (4500): 4500
After filtering: 133 records remain

Available Points of Interest:
1. fire
2. police
3. early_childhood_school
4. elementary_school
5. middle_school
6. high_school
7. hospital
8. nursing_home
9. public_health
10. urgentcare
11. trailheads

Select a POI (1-11) or 0 to finish: 7
How important is it for this POI (hospital) to be CLOSE to the property? (1-5): 5
Added hospital

Select a POI (1-11) or 0 to finish: 11
How important is it for this POI (trailheads) to be CLOSE to the property? (1-5): 4
Added trailheads

Select a POI (1-11) or 0 to finish: 0

===== PROPERTIES RANKED BY TOPSIS SCORE =====
                 address  house_price  bedrooms  bathrooms  square